# Imports #

In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from decouple import config
import requests
import urllib.parse
import json
AAapikey = config('AAKey')

# Gather Historic Stock News #

In [2]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)

In [3]:
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1), interval = 365):
    # Format for news is [Date, Headline, Ticker]
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    url = 'https://www.alphavantage.co/query?'
    window = 15
    startDate = endDate - datetime.timedelta(days = interval)
    windowStart = startDate
    windowEnd = startDate + datetime.timedelta(days = window)
    delta = datetime.timedelta(days = window)
    count = 0
    while (windowEnd <= endDate):
        if count == 5:
            time.sleep(60)
            count = 0
        startDateAA = toAADate(windowStart)
        endDateAA = toAADate(windowEnd)
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDateAA, 'time_to': endDateAA, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
        r = requests.get(url, params = Myparams)
        data = r.json()
        try:
            for i in data.get("feed"):
                test_date = i.get("time_published")
                test_date = test_date[:8]
                newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date() 
                row = [newDate, i.get("title"), ticker]
                new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
                historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
        except:
            print("Request failed")
            print("windowStart is: " +str(windowStart))
            print("windowEnd is: " +str(windowEnd))
            print(data)
        windowStart = windowEnd
        windowEnd = windowEnd + delta
        count = count + 1
    print("Final rolling date is: " +str(windowEnd))
    #print(historic_parsed_news)
    return historic_news


In [4]:

ticker = input('Input Ticker:\n')  
print(ticker)

UserInt = input('Input Interval: \n')
print(UserInt)

AAPL


In [5]:
hist = getHistoricNewsData(ticker, interval = UserInt)

Request failed
windowStart is: 2020-02-22
windowEnd is: 2020-03-08
{'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}
Request failed
windowStart is: 2020-03-08
windowEnd is: 2020-03-23
{'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}
Request failed
windowStart is: 2020-03-23
windowEnd is: 2020-04-07
{'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}
Request failed
windowStart is: 2020-04-07
windowEnd is: 2020-04-22
{'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}
Request failed
windowStart is: 2020-04-22
windowEnd is: 2020-05-07
{

In [6]:
print(hist.head)

<bound method NDFrame.head of             Date                                           Headline Ticker
0     2022-03-01  US stocks fall, oil tops $105 as Ukraine crisi...   AAPL
1     2022-03-02  Apple halts product sales in Russia after Ukra...   AAPL
2     2022-03-02  Russia says its economy is taking 'serious blo...   AAPL
3     2022-03-02  Rich Russians turn to luxury jewellery, watche...   AAPL
4     2022-03-03  Market Rallies, But Still Must Do This; 5 Stoc...   AAPL
...          ...                                                ...    ...
4795  2023-02-13  SPY: Diversify Your Portfolio in 2023 With The...   AAPL
4796  2023-02-13  10 Stocks That Are Crushing Short Sellers In 2...   AAPL
4797  2023-02-13  Why Steve Wozniak Thinks ChatGPT Can Never Rep...   AAPL
4798  2023-02-13  Pressure Sensor Market Report will Achieve USD...   AAPL
4799  2023-02-13              1 Billion Reasons to Love Apple Stock   AAPL

[4800 rows x 3 columns]>


In [7]:
histname = "datasets/" + ticker + "-test-hist.csv"
hist.to_csv(histname)